In [ ]:
import sys
import numpy as np
import pandas as pd
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
import random
import scipy.integrate
import scipy.stats
from sklearn import linear_model
################################
from helper_functions import z_expectation_variance, KMS_Matrix, product_diff_list, moment_matching_update, g_fun, g_fun_linear_regression, question_selection_prob, g_opt, two_step_g_acq, question_extractor, rollout,monte_carlo_rollout, batch_design_delta_penalty, coordinate_exchange_acq, enum_two_step, enum_two_step_opt, rollout_with_batch_design_acquisition

In [ ]:
#This function is used to compare different methods in a sequential fashion using normalized MSE, determinant, 
#hitrate, and MSE as a measurement 
#of quality. In this function, we only use one acquisition method at a time.

def new_sequential_experiment(init_mu, init_Sig, true_partworths, rep_per_partworth, num_questions,look_ahead_horizon, mu_log_coeff,
                                   Sig_log_coeff,noise_par, question_list, Method = 0,
                                   batch_size = 0, MC_budget = 0, include_one_step = False,penalty = 100,rel_gap = 0.0001):
    #init_mu: This is the initial estimate on the partworths
    #init_Sig: This is the initial covariance matrix on the partworths
    #true_partworths: These are used to make selection in the product selection stage (a list/set of partworths)
    #rep_per_partworth: This is the number of times we want to conduct a questionnaire on each partworth
    #num_questions: Length of the questionnaire
    #look_ahead_horizon: This is the number of stages that we look ahead in a rollout method.
    #mu_log_coeff; Sig_log_coeff: These are the coefficients in the optimization model for the linearized objective function
    #noise_par: This is a parameter which is used to increase the weight of the individuals' true partworth when making decision
    #question_list: This is a list of questions which will be used to calculate the hitrate, which is the
    #proportion of times that an estimated partworth matches the product selection of a true underlying partworth.
    #between x and y. The higher this weight is, the less effect the gumbel random variable has on user choice.
    #Method: 0 - One step look ahead
    #        1 - Rollout with batch design   <------ CAN ADD MORE METHODS IF NEEDED
    #        2 - Two step look ahead via enumeration
    #        3 - Rollout using coordinate exchange
    #batch_size: Used to select batch size if we use rollout
    #        OPTIONS:
    #        batch_size = n (n is a non-negative integer): This will make it so that the batch size is constant and equal to n for every rollout iteration
    #        batch_size = -1: This will make it so that the batch size is equal to the look-ahead horizon
    #MC_budget: Used to select Monte Carlo budget if we use rollout.
    #include_one_step: This determines whether we want to include the one-step optimal question within our batch. This can
    #help ensure that rollout performs at least as well as one-step look ahead. Default value is False.
    #penalty_term: This is used to set the penalty level for orthogonality in the orthogonal batch design optimization problem. A higher penalty
    #term will lead to a more Sigma_orthogonal design, while a lower penalty term will lead to less Sigma_orthogonality in the design
    #rel_gap: This is used in rollout with coordinate exchange. This value measures the improvement of a question's rollout
    #value over the current question's rollout value. If the improvement is greater than the relative gap, we update the current
    #question
    
    
    #Construct lists for storing normalized MSE, determinant, hitrate, and MSE information. 
    #For each of the baseline partworths, we create a list holding
    #(num_questions) lists, where each of the (num_questions) lists holds the MSE and determinant values for all replications
    #after the first, second,...,nth question
    
    num_true_partworth = len(true_partworths) 
    
    hitrate_total_num_of_questions = len(question_list)
    
    
    #Construct the expected preferred product selection for each true partworth. This will be used when we calculate the probability
    #of correct selection
    x_trueselection = [[] for u in range(num_true_partworth)]
    attributes_num = len(init_mu)
    for u in range(num_true_partworth):
        x_trueselection[u] = np.array([1.0 if partworth_component>=0.0 else 0.0 for partworth_component in 
                                           true_partworths[u]])
    
    #Set up lists to hold normalized MSE, determinant, MSE, and hitrate information. For now, we will only calculate hitrate at the end
    #of the questionnaire. Also save the mu vectors. 
    
    MSE_normalized = [[[] for j in range(num_questions)] for u in range(num_true_partworth)]
    
    DET = [[[] for j in range(num_questions)] for u in range(num_true_partworth)]
    
    HITRATE = [[] for u in range(num_true_partworth)]
    
    MSE = [[[] for j in range(num_questions)] for u in range(num_true_partworth)]
    
    PROB_CORRECT_SEL = []
    
    MU = [[[] for j in range(num_questions)] for u in range(num_true_partworth)]
    
    for u in range(num_true_partworth):
        #create a variable which will store the number of correct selections.
        correct_sel = 0
        for i in range(rep_per_partworth):
            #Instantiate mu and Sig with the initial parameters init_mu and init_Sig. These act as prior parameters for all
            #the partworths
            mu = init_mu
            Sig = init_Sig
            for j in range(num_questions):
                if Method == 0:
                    #get optimal question for one step
                    [x,y] = g_opt(mu,Sig,mu_log_coeff,Sig_log_coeff)[1:]
                    
                if Method == 1:
                    #get optimal question for rollout over batch design
                    #Use a rollout length equal to look_ahead_horizon - 1, until we are within look_ahead_horizon of the
                    #budget
                    
                    if j < num_questions - look_ahead_horizon:
                        rollout_len = look_ahead_horizon - 1
                    else:
                        rollout_len = num_questions - j - 1
                    
                    if batch_size == -1:
                        #When batch_size is -1, we use the rollout length + 1 as the batch size.
                        [x,y] = rollout_with_batch_design_acquisition(mu,Sig,mu_log_coeff,Sig_log_coeff,rollout_len+1,
                                                                 rollout_len,MC_budget,include_one_step,penalty_term = penalty)
                    else:
                        [x,y] = rollout_with_batch_design_acquisition(mu,Sig,mu_log_coeff,Sig_log_coeff,batch_size,
                                                                 rollout_len,MC_budget,include_one_step,penalty_term = penalty)
                    
                if Method == 2:
                    #Do the enumeration procedure for two-step look ahead
                    if j < num_questions-1:
                        enumerated_solution_list_two_step = enum_two_step(mu,Sig,mu_log_coeff,
                                                             Sig_log_coeff,question_list)
                        opt_sol_questions_two_step = enum_two_step_opt(enumerated_solution_list_two_step[0],
                                                          enumerated_solution_list_two_step[2])
                    
                        #get the first stage products for true two-step
                        x = np.array(opt_sol_questions_two_step[1])
                        y = np.array(opt_sol_questions_two_step[2])
                    #Use one-step for the last question.
                    else:
                        [x,y] = g_opt(mu,Sig,mu_log_coeff,Sig_log_coeff)[1:]
                    
                if Method == 3:
                    #Do rollout with coordinate exchange
                    if j < num_questions - look_ahead_horizon:
                        rollout_len = look_ahead_horizon - 1
                    else:
                        rollout_len = num_questions - j - 1
                        
                    [x,y] = coordinate_exchange_acq(mu,Sig,mu_log_coeff,Sig_log_coeff,batch_size,rollout_len,
                                                   MC_budget,rel_gap,include_batch = False,include_one_step = True)
                
                #!!!WE WILL NOT USE GUMBEL RANDOM VARIABLE IN TYPE II EXPERIMENT!!!
                #Instantiate gumbel random variables which are used in the product choice selection process.
                #gum_x = rng.gumbel(0,1,1)
                #gum_y = rng.gumbel(0,1,1)
                    
                #These temp variables will be used in the choice model below in case the user prefers y over x.
                x_temp = x
                y_temp = y
                    
                #See preference between two products
                #set signal to noise ratio
                if (noise_par*np.dot(true_partworths[u],np.array(y))) >= (noise_par*np.dot(true_partworths[u],np.array(x))):
                    x = y_temp
                    y = x_temp
                
                #Perform moment matching after choice is made.
                [mu, Sig] = moment_matching_update(x,y,mu,Sig)
                
                #add mu to the list of mu vectors
                MU[u][j].append(mu)
                #Add the normalized MSE between the true partworth and estimator at question j to a list, and add the determinant of
                #the covariance matrix at question j into a list. Also add the regular MSE
                MSE_normalized[u][j].append(np.square(np.subtract(true_partworths[u]/np.linalg.norm(true_partworths[u],ord = 2),
                                                            mu/np.linalg.norm(mu, ord = 2))).mean())
                DET[u][j].append(np.linalg.det(Sig))
                
                MSE[u][j].append(np.square(np.subtract(true_partworths[u],
                                                            mu)).mean())
                
            #Calculate hitrate for this replication. Given a set of questions of length K, we compare how well the final 
            #estimator performs in terms of correctly selecting the preferred profile for each question. The estimator
            #makes a correct selection if its selection matches that of the true partworth (selection is absent of gumble noise)
            hits = 0
            for q in question_list:
                if np.dot(true_partworths[u],q)*np.dot(mu,q)>=0:
                    hits = hits + 1
            HITRATE[u].append(hits/hitrate_total_num_of_questions)
            
            #Calculate whether we have correct selection in this replication for estimator mu
            x_mu = np.array([1.0 if mu_component>=0.0 else 0.0 for mu_component in 
                                           mu])
            if np.dot(x_trueselection[u]-x_mu,x_trueselection[u]-x_mu) == 0.0:
                correct_sel = correct_sel + 1
                
        
        #Calculate the probability of correct selection for the current true partworth
        PROB_CORRECT_SEL.append(correct_sel/rep_per_partworth)
                
        
    return[MSE_normalized,DET,HITRATE,MSE,PROB_CORRECT_SEL,true_partworths,MU]

In [ ]:
#This function is used to define the experimental setting that will be used in the sequential experiment. The experimental 
#settings are found in the LaTex document "Bayesian Sequential Preference Learning Experimental Settings"
def experiment_settings(attr,bp_exp,bp_cov,signal_noise,look_ahead,ortho_pen):
    #attr: The number of attributes of a product. Note that the rest of the 
        #1 : 6 attributes
        #2 : 12 attributes
    #bp_exp: This is the type of bayesian prior expectation we start with
        #1 : homogeneous
        #2 : heterogeneous
    #bp_cov: This the the type of bayesian prior covariance we start with
        #1 : homogeneous diagonal
        #2 : heterogeneous diagonal
        #3 : non-diagonal
    #signal_noise_ratio: This is the signal to noise ratio between the expectation and covariance.
        #1 : Low = 0.25
        #2 : Normal = 1.0
        #3 : High = 4.0
    #look_ahead: This is the look ahead horizon that we will use
        #1 : 1/3 of the number of attributes
        #2 : 2/3 of the number of attributes
        #3 : Equal to the number of attributes
    #ortho_pen: This is the parameter which penalizes the orthogonality condition in the orthogonal batch design
        #1 : small = 0.01
        #2 : large = 10.0
    
    
    if attr == 1:
        #Define number of attributes
        attributes = 6
        
        #define bayesian prior parameters
        if bp_exp == 1:
            bayes_expectation = np.array(attributes*[1.0])
        if bp_exp == 2:
            bayes_expectation = np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        
        
        if bp_cov == 1:
            bayes_covariance = np.identity(attributes)
        if bp_cov == 2:
            bayes_covariance = np.diag([2.0,1/2,4.0,1/4,8.0,1/8])
        if bp_cov == 3:
            bayes_covariance = 1.25*KMS_Matrix(attributes,-0.5)
        
        if look_ahead == 1:
            look_ahead_horizon = 2
        if look_ahead == 2:
            look_ahead_horizon = 4
        if look_ahead == 3:
            look_ahead_horizon = 6
        
        #calculate coefficient parameters for optimization problem
        mu_log,Sig_log = g_fun_linear_regression(0,12.70,0.1,42.0,24,84)
        
        #Create a list of questions
        question_list = product_diff_list(attributes)
        
    if attr == 2:
        #Define number of attributes
        attributes = 12
        
        #Define bayesian prior parameters
        if bp_exp == 1:
            bayes_expectation = np.array(attributes*[1.0])
        if bp_exp == 2:
            bayes_expectation = np.array([-0.125,0.25,-0.375,0.5,-0.625,0.75,-0.875,1.0,
                                         -1.125,1.25,-1.375,1.5])
        
        if bp_cov == 1:
            bayes_covariance = np.identity(attributes)
        if bp_cov == 2:
            bayes_covariance = np.diag([2.0,1/2,3.0,1/3,4.0,1/4,16/3,3/16,20/3,
                                       3/20,8,1/8])
        if bp_cov == 3:
            bayes_covariance = 1.25*KMS_Matrix(attributes,-0.5)
        
        if look_ahead == 1:
            look_ahead_horizon = 4
        if look_ahead == 2:
            look_ahead_horizon = 8
        if look_ahead == 3:
            look_ahead_horizon = 12
        
        #calculate coefficient parameters for optimization problem
        mu_log,Sig_log = g_fun_linear_regression(0, 24.50, 0.1, 156.0, 49, 312)
        
        #Create a list of questions, take a random sample of 3^6 since there are around 3^12 possibilities
        question_list = product_diff_list(attributes)
    
        question_list = random.sample(question_list,3**6)
    
    #Set up signal to noise ratio
    if signal_noise == 1:
        signal_noise_ratio = 0.25
    if signal_noise == 2:
        signal_noise_ratio = 1.0
    if signal_noise == 3:
        signal_noise_ratio = 4.0
        
    #Set up orthogonality parameter
    if ortho_pen == 1:
        orthogonality_penalty_parameter = 0.01
    if ortho_pen == 2:
        orthogonality_penalty_parameter = 10.0
    
    #Scale the bayes expectation by the signal to noise ratio   
    bayes_scale_expectation = signal_noise_ratio*bayes_expectation
    
    return[attributes,bayes_scale_expectation,bayes_covariance,look_ahead_horizon,
          orthogonality_penalty_parameter,mu_log,Sig_log,question_list]

In [ ]:
#Set up some parameters. 

#Set up random number seed
rng = np.random.default_rng(100) 
np.random.seed(100)
random.seed(100)

#EXPERIMENT SETTING ARGUMENTS
attr_exp = int(sys.argv[1])
bp_expect_exp = int(sys.argv[2])
bp_cov_exp = int(sys.argv[3])
signal_noise_exp = int(sys.argv[4])
look_ahead_exp = int(sys.argv[5])
ortho_pen_exp = int(sys.argv[6])


#We need a way to read in the arguments for experiment_settings (for job arrays)
[attributes_exp,bayes_expectation_exp,bayes_covariance_exp,look_ahead_horizon_exp,orthogonality_penalty_parameter_exp,
mu_log_exp,Sig_log_exp,question_list_exp] = experiment_settings(attr_exp,bp_expect_exp,bp_cov_exp,signal_noise_exp,
                                                               look_ahead_exp,ortho_pen_exp)

#print([bayes_expectation_exp,bayes_covariance_exp,look_ahead_horizon_exp,orthogonality_penalty_parameter_exp,
#mu_log_exp,Sig_log_exp,question_list_exp])
#Create list of true partworths. This will have the property that one is close to the initial estimator, and one is 
#far away
samp_num_partworths = 100

#Create a list of random partworths
partworths_exp = []

for i in range(0,samp_num_partworths):
        partworths_exp.append(rng.multivariate_normal(bayes_expectation_exp,bayes_covariance_exp))

#!!!WE DO NOT USE THE MAX AND MIN MSE VECTORS!!!
#Use only the two partworths with the max and min MSE compared to the initial parameter estimator.
#init_mse_values = samp_num_partworths*[0.0]
#for i in range(0,samp_num_partworths):
    #init_mse_values[i] =  np.square(np.subtract(partworths_exp[i]/np.linalg.norm(partworths_exp[i],ord = 2),
                                                            #bayes_expectation_exp/np.linalg.norm(bayes_expectation_exp, ord = 2))).mean()
#init_mse_max_index = np.argmax(np.array(init_mse_values))
#init_mse_min_index = np.argmin(np.array(init_mse_values))
#partworths_exp = [partworths_exp[init_mse_min_index],partworths_exp[init_mse_max_index]]

#repetition per partworth
rep_per_partworth_exp = 1

#number of questions
num_questions_exp = 16

#noise parameter in selection process
noise_par_exp = 1.0

#batch size
batch_size_exp = -1

#Monte Carlo sampling budget
MC_budget_exp = 50


In [ ]:
#ONESTEP
#Set up random number seed
rng = np.random.default_rng(100) 
np.random.seed(100)
random.seed(100)

[Norm_MSE_OS,DET_OS,HITRATE_OS,MSE_OS,Prob_sel_OS,true_partworths_OS,MU_OS] = new_sequential_experiment(bayes_expectation_exp,bayes_covariance_exp,partworths_exp,rep_per_partworth_exp,num_questions_exp,
                                           look_ahead_horizon_exp,mu_log_exp,Sig_log_exp,noise_par_exp,question_list_exp,Method = 0,
                                            batch_size = batch_size_exp,MC_budget = MC_budget_exp,include_one_step = True,
                                                                penalty = orthogonality_penalty_parameter_exp, rel_gap = 0.0001)


In [ ]:
#Rollout with batch design
#Set up random number seed
rng = np.random.default_rng(100) 
np.random.seed(100)
random.seed(100)

[Norm_MSE_RB,DET_RB,HITRATE_RB,MSE_RB,Prob_sel_RB,true_partworths_RB,MU_RB] = new_sequential_experiment(bayes_expectation_exp,bayes_covariance_exp,partworths_exp,rep_per_partworth_exp,num_questions_exp,
                                           look_ahead_horizon_exp,mu_log_exp,Sig_log_exp,noise_par_exp,question_list_exp,Method = 1,
                                            batch_size = batch_size_exp,MC_budget = MC_budget_exp,include_one_step = True,
                                                                penalty = orthogonality_penalty_parameter_exp, rel_gap = 0.0001)

In [ ]:
#Only do two_step when there are 6 attributes.
if attr_exp == 1:
    #TWOSTEP via Enumeration
    #Set up random number seed
    rng = np.random.default_rng(100) 
    np.random.seed(100)
    random.seed(100)

    [Norm_MSE_TS,DET_TS,HITRATE_TS,MSE_TS,Prob_sel_TS,true_partworths_TS,MU_TS] = new_sequential_experiment(bayes_expectation_exp,bayes_covariance_exp,partworths_exp,rep_per_partworth_exp,num_questions_exp,
                                               look_ahead_horizon_exp,mu_log_exp,Sig_log_exp,noise_par_exp,question_list_exp,Method = 2,
                                                batch_size = batch_size_exp,MC_budget = MC_budget_exp,include_one_step = True,
                                                                    penalty = orthogonality_penalty_parameter_exp, rel_gap = 0.0001)

In [ ]:
#Rollout with coordinate exchange
#Set up random number seed
rng = np.random.default_rng(100) 
np.random.seed(100)
random.seed(100)

[Norm_MSE_RC,DET_RC,HITRATE_RC,MSE_RC,Prob_sel_RC,true_partworths_RC,MU_RC] = new_sequential_experiment(bayes_expectation_exp,bayes_covariance_exp,partworths_exp,rep_per_partworth_exp,num_questions_exp,
                                           look_ahead_horizon_exp,mu_log_exp,Sig_log_exp,noise_par_exp,question_list_exp,Method = 3,
                                            batch_size = batch_size_exp,MC_budget = MC_budget_exp,include_one_step = True,
                                                                penalty = orthogonality_penalty_parameter_exp, rel_gap = 0.0001)

In [ ]:
#Construct the columns of the dataframe.

#Collect the data above into one list. Note that we will not have two step when there are 12 attributes due to enumeration
#taking too long.
if attr_exp == 1:
    data_collection = [[Norm_MSE_OS,DET_OS,HITRATE_OS,MSE_OS,Prob_sel_OS,true_partworths_OS,MU_OS],[Norm_MSE_RB,DET_RB,HITRATE_RB,MSE_RB,Prob_sel_RB,true_partworths_RB,MU_RB],
                       [Norm_MSE_TS,DET_TS,HITRATE_TS,MSE_TS,Prob_sel_TS,true_partworths_TS,MU_TS],[Norm_MSE_RC,DET_RC,HITRATE_RC,MSE_RC,Prob_sel_RC,true_partworths_RC,MU_RC]]
    method_range = 4

if attr_exp == 2:
    data_collection = [[Norm_MSE_OS,DET_OS,HITRATE_OS,MSE_OS,Prob_sel_OS,true_partworths_OS,MU_OS],[Norm_MSE_RB,DET_RB,HITRATE_RB,MSE_RB,Prob_sel_RB,true_partworths_RB,MU_RB],
                       [Norm_MSE_RC,DET_RC,HITRATE_RC,MSE_RC,Prob_sel_RC,true_partworths_RC,MU_RC]]
    method_range = 3

Norm_MSE_col = []
DET_col = []
MSE_col = []
HITRATE_col = []
Prob_sel_col = []
Part_col = []
Method_col = []

rep_col = []
quest_col = []

#Columns here are the entries in the ith component of the true partworth and estimator mu
true_partworths_col = [[] for i in range(attributes_exp)]
MU_col = [[] for i in range(attributes_exp)]

#Construct the Norm_MSE, DET, and MSE columns
#Methods are 0=OS, 1=RB, 2=TS, 3=RC for 6 attributes. 0=OS, 1=RB, 2=RC for 12 attributes

for Meth in range(method_range):
    for part in range(100):
        for r in range(rep_per_partworth_exp):
            for q in range(num_questions_exp):
                Norm_MSE_col.append(data_collection[Meth][0][part][q][r])
                DET_col.append(data_collection[Meth][1][part][q][r])
                MSE_col.append(data_collection[Meth][3][part][q][r])
                
                HITRATE_col.append(data_collection[Meth][2][part][r])
                Prob_sel_col.append(data_collection[Meth][4][part])
                
                Method_col.append(Meth)
                Part_col.append(part)
                
                rep_col.append(r + 1)
                quest_col.append(q + 1)
                
                for i in range(attributes_exp):
                    true_partworths_col[i].append(data_collection[Meth][5][part][i])
                    MU_col[i].append(data_collection[Meth][6][part][q][r][i])

#Data for the mu vector and true partworths
df_columns_data = [Method_col,Part_col,rep_col,quest_col,Norm_MSE_col,DET_col,MSE_col,HITRATE_col,Prob_sel_col]

for i in range(attributes_exp):
    df_columns_data.append(MU_col[i])

for i in range(attributes_exp):
    df_columns_data.append(true_partworths_col[i])
    
df_columns_data = np.array(df_columns_data).T

#Make column names
df_columns_names = ['Method', 'Partworth','Rep','Question','Norm_MSE','Det','MSE','Hitrate','Prob. Sel']

for i in range(attributes_exp):
    df_columns_names.append('Mu_' + str(i+1))


for i in range(attributes_exp):
    df_columns_names.append('True_' + str(i+1))
    
#Make dataframe. The number of columns is 9 + 2*attributes. The number of rows is method_range*100*rep_per_partworth*num_questions
#100 comes from the 100 true_partworths that we use.
df_data = pd.DataFrame(data = df_columns_data, index = range(1,method_range*100*rep_per_partworth_exp*num_questions_exp + 1),
                      columns = df_columns_names)
#print(df_data)
#df_data.to_csv(r'C:\Users\wsfishe\Desktop\df_data.csv',index=True,header=True)
df_data.to_csv('\sequential_data_TYPEII_attr_'+str(attr_exp)+ '_exp_'+str(bp_expect_exp)+'_cov_'+str(bp_cov_exp) +
               '_snr_' + str(signal_noise_exp) + '_look_' + str(look_ahead_exp) + '_orth_' + str(ortho_pen_exp)+'.csv',
               index=True,header=True)

#print(df_data)